## **Federated Learning for attack detection: 3 nodes sharing gradients**

IDs from this file = **id6xy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [2]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [4]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [5]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

### Experiments with datasets

#### Dataset 3A

In [6]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3A-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_8972/4261398225.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3A-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8972/4261398225.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8972/4261398225.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8972/4261398225.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or s

In [7]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [8]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id600.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [9]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [10]:
global_model = build_model((24,1,1))

In [11]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [12]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [13]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id600.hdf5')

Epoch 1/5
51/51 [==============================] - 13s 247ms/step - loss: 0.2771 - accuracy: 0.9762 - val_loss: 3.3810 - val_accuracy: 0.2939
Epoch 2/5
51/51 [==============================] - 13s 249ms/step - loss: 0.0355 - accuracy: 0.9913 - val_loss: 1.2808 - val_accuracy: 0.3233
Epoch 3/5
51/51 [==============================] - 13s 250ms/step - loss: 0.0271 - accuracy: 0.9917 - val_loss: 1.1547 - val_accuracy: 0.3412
Epoch 4/5
51/51 [==============================] - 13s 252ms/step - loss: 0.0254 - accuracy: 0.9917 - val_loss: 1.3692 - val_accuracy: 0.2977
Epoch 5/5
51/51 [==============================] - 11s 223ms/step - loss: 0.0238 - accuracy: 0.9921 - val_loss: 0.7375 - val_accuracy: 0.5239
Epoch 1/5
51/51 [==============================] - 12s 242ms/step - loss: 0.0205 - accuracy: 0.9931 - val_loss: 0.1361 - val_accuracy: 0.9190
Epoch 2/5
51/51 [==============================] - 11s 219ms/step - loss: 0.0173 - accuracy: 0.9942 - val_loss: 0.1491 - val_accuracy: 0.9232
Epoch 

In [14]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.269886314868927, 0.38635921478271484], [0.1566026210784912, 0.36180034279823303], [0.2322198897600174, 0.7285167574882507]]
Accuracy for iterations:  [[0.9331414699554443, 0.8716123104095459], [0.9866487383842468, 0.8541197180747986], [0.9502395391464233, 0.7682191133499146]]
F1 for iterations:  [[0.9327762071646682, 0.8681414449368922], [0.9866519742811942, 0.8494644306326714], [0.9500586250958318, 0.7487386917147426]]
Precision for iterations:  [[0.9372113567617012, 0.8882074512071244], [0.9866960188109921, 0.873779660181913], [0.9526364067437322, 0.8248187420742665]]
Recall for iterations:  [[0.9331414882117502, 0.8716122938312868], [0.9866487481375442, 0.8541197413354646], [0.9502395629437027, 0.7682191382692727]]


#### Dataset 3B 

In [15]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3B-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_8972/1530530892.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3B-Part2.csv')


In [16]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [17]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id601.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [18]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [19]:
global_model = build_model((24,1,1))

In [20]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [21]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [22]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id601.hdf5')

Epoch 1/5
51/51 [==============================] - 11s 211ms/step - loss: 0.1292 - accuracy: 0.9992 - val_loss: 1.7152e-05 - val_accuracy: 1.0000
Epoch 2/5
51/51 [==============================] - 11s 213ms/step - loss: 1.1471e-05 - accuracy: 1.0000 - val_loss: 9.7977e-06 - val_accuracy: 1.0000
Epoch 3/5
51/51 [==============================] - 12s 226ms/step - loss: 9.3926e-06 - accuracy: 1.0000 - val_loss: 8.8217e-06 - val_accuracy: 1.0000
Epoch 4/5
51/51 [==============================] - 12s 237ms/step - loss: 8.4525e-06 - accuracy: 1.0000 - val_loss: 7.9229e-06 - val_accuracy: 1.0000
Epoch 5/5
51/51 [==============================] - 11s 224ms/step - loss: 7.5693e-06 - accuracy: 1.0000 - val_loss: 7.0754e-06 - val_accuracy: 1.0000
Epoch 1/5
51/51 [==============================] - 8s 151ms/step - loss: 1.0974 - accuracy: 0.6250 - val_loss: 0.8066 - val_accuracy: 0.0000e+00
Epoch 2/5
51/51 [==============================] - 8s 150ms/step - loss: 0.3303 - accuracy: 0.8723 - val_loss

In [23]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[4.998519420623779, 5.220028400421143], [5.632476806640625, 5.969873428344727], [4.890505790710449, 5.1775360107421875]]
Accuracy for iterations:  [[0.471259742975235, 0.4393664300441742], [0.4713546931743622, 0.4394209086894989], [0.4717783033847809, 0.4397297203540802]]
F1 for iterations:  [[0.30195648712973155, 0.26837947544156576], [0.3021657967644099, 0.268498556625407], [0.3030988407521974, 0.2691729649829838]]
Precision for iterations:  [[0.7508402958425918, 0.753716401752156], [0.7508613821852271, 0.7537269185756607], [0.7509555085271363, 0.7537865355700379]]
Recall for iterations:  [[0.47125975050395863, 0.43936641720555114], [0.4713546992316457, 0.4394209111385599], [0.4717783166320956, 0.4397297100922764]]


#### Dataset 3C

In [24]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3C-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_8972/2150709242.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3C-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8972/2150709242.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3C-Part3.csv')


In [25]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [26]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id602.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [27]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [28]:
global_model = build_model((24,1,1))

In [29]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [30]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [31]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id602.hdf5')

Epoch 1/5
26/26 [==============================] - 5s 174ms/step - loss: 0.3379 - accuracy: 0.9630 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 2/5
26/26 [==============================] - 4s 164ms/step - loss: 0.0180 - accuracy: 0.9988 - val_loss: 1.0095e-04 - val_accuracy: 1.0000
Epoch 3/5
26/26 [==============================] - 4s 169ms/step - loss: 0.0138 - accuracy: 0.9988 - val_loss: 0.0029 - val_accuracy: 1.0000
Epoch 4/5
26/26 [==============================] - 4s 161ms/step - loss: 0.0081 - accuracy: 0.9988 - val_loss: 0.0347 - val_accuracy: 1.0000
Epoch 5/5
26/26 [==============================] - 5s 188ms/step - loss: 0.0042 - accuracy: 0.9988 - val_loss: 0.0380 - val_accuracy: 0.9986
Epoch 1/5
51/51 [==============================] - 8s 164ms/step - loss: 0.5802 - accuracy: 0.8547 - val_loss: 1.3953 - val_accuracy: 0.6150
Epoch 2/5
51/51 [==============================] - 9s 180ms/step - loss: 0.0658 - accuracy: 0.9913 - val_loss: 1.0074 - val_accuracy: 0.6173
Epoch 3/5

In [32]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.06623820215463638, 0.2545536756515503], [0.06610514223575592, 0.2335139662027359], [0.08103546500205994, 0.21621164679527283]]
Accuracy for iterations:  [[0.9823906421661377, 0.87822425365448], [0.9847278594970703, 0.9045266509056091], [0.9805939197540283, 0.921565055847168]]
F1 for iterations:  [[0.9823851797095589, 0.875168327832335], [0.9847269640137685, 0.9030180032236031], [0.9805860543248166, 0.9207001222559648]]
Precision for iterations:  [[0.9824282326610256, 0.8934943786819013], [0.9847280443740247, 0.9128781556696317], [0.9806571853604926, 0.9266130258528362]]
Recall for iterations:  [[0.9823906628881942, 0.8782242243696868], [0.9847278623389524, 0.904526629368597], [0.980593940810424, 0.9215650657560125]]


#### Dataset 3D

In [33]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3D-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3D-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3D-Part3.csv')

In [34]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [35]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id603.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [36]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [37]:
global_model = build_model((24,1,1))

In [38]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [39]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [40]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])



global_model.save('C:/Users/UX430/Documents/thesis/code/models/id603.hdf5')

Epoch 1/5
13/13 [==============================] - 3s 199ms/step - loss: 0.4505 - accuracy: 0.9962 - val_loss: 0.1225 - val_accuracy: 1.0000
Epoch 2/5
13/13 [==============================] - 2s 180ms/step - loss: 0.0318 - accuracy: 1.0000 - val_loss: 9.9241e-04 - val_accuracy: 1.0000
Epoch 3/5
13/13 [==============================] - 2s 155ms/step - loss: 3.2011e-04 - accuracy: 1.0000 - val_loss: 6.0621e-05 - val_accuracy: 1.0000
Epoch 4/5
13/13 [==============================] - 2s 167ms/step - loss: 3.7339e-05 - accuracy: 1.0000 - val_loss: 2.2484e-05 - val_accuracy: 1.0000
Epoch 5/5
13/13 [==============================] - 2s 169ms/step - loss: 1.9396e-05 - accuracy: 1.0000 - val_loss: 1.6396e-05 - val_accuracy: 1.0000
Epoch 1/5
13/13 [==============================] - 2s 169ms/step - loss: 5.5540 - accuracy: 0.0000e+00 - val_loss: 2.3688 - val_accuracy: 0.0000e+00
Epoch 2/5
13/13 [==============================] - 2s 155ms/step - loss: 1.3972 - accuracy: 0.0000e+00 - val_loss: 0.7

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1721/1721 [==============================] - 5s 3ms/step - loss: 6.6773 - accuracy: 0.4392


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
13/13 [==============================] - 2s 167ms/step - loss: 7.6749 - accuracy: 0.0000e+00 - val_loss: 3.8253 - val_accuracy: 0.0000e+00
Epoch 2/5
13/13 [==============================] - 2s 155ms/step - loss: 2.1286 - accuracy: 1.5063e-04 - val_loss: 0.9624 - val_accuracy: 0.0011
Epoch 3/5
13/13 [==============================] - 2s 154ms/step - loss: 0.6069 - accuracy: 0.6554 - val_loss: 0.2978 - val_accuracy: 1.0000
Epoch 4/5
13/13 [==============================] - 2s 152ms/step - loss: 0.1620 - accuracy: 1.0000 - val_loss: 0.0588 - val_accuracy: 1.0000
Epoch 5/5
13/13 [==============================] - 2s 161ms/step - loss: 0.0314 - accuracy: 1.0000 - val_loss: 0.0130 - val_accuracy: 1.0000
Epoch 1/5
13/13 [==============================] - 2s 188ms/step - loss: 1.6333 - accuracy: 0.0000e+00 - val_loss: 0.7891 - val_accuracy: 0.0000e+00
Epoch 2/5
13/13 [==============================] - 2s 148ms/step - loss: 0.5555 - accuracy: 0.8371 - val_loss: 0.3660 - val_accuracy: 

...

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
13/13 [==============================] - 2s 154ms/step - loss: 9.1337 - accuracy: 0.0000e+00 - val_loss: 5.1446 - val_accuracy: 0.0000e+00
Epoch 2/5
13/13 [==============================] - 2s 154ms/step - loss: 2.6841 - accuracy: 0.1114 - val_loss: 0.2995 - val_accuracy: 1.0000
Epoch 3/5
13/13 [==============================] - 2s 148ms/step - loss: 0.0613 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 4/5
13/13 [==============================] - 2s 147ms/step - loss: 6.8383e-04 - accuracy: 1.0000 - val_loss: 2.3278e-04 - val_accuracy: 1.0000
Epoch 5/5
13/13 [==============================] - 2s 152ms/step - loss: 1.7309e-04 - accuracy: 1.0000 - val_loss: 1.2766e-04 - val_accuracy: 1.0000
Epoch 1/5
13/13 [==============================] - 2s 187ms/step - loss: 2.9450 - accuracy: 0.1630 - val_loss: 0.2792 - val_accuracy: 1.0000
Epoch 2/5
13/13 [==============================] - 2s 145ms/step - loss: 0.0924 - accuracy: 1.0000 - val_loss: 0.0208 - val_accura

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1721/1721 [==============================] - 6s 3ms/step - loss: 8.3740 - accuracy: 0.4392: 0s - loss: 7.8252 


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[6.378147125244141, 6.677319049835205], [7.572839260101318, 7.96567964553833], [7.941164493560791, 8.37397575378418]]
Accuracy for iterations:  [[0.4712086319923401, 0.4392392635345459], [0.4712086319923401, 0.4392392635345459], [0.4712086319923401, 0.4392392635345459]]
F1 for iterations:  [[0.30184375474744357, 0.2681015400046588], [0.30184375474744357, 0.2681015400046588], [0.30184375474744357, 0.2681015400046588]]
Precision for iterations:  [[0.2220375675826312, 0.19293113164997755], [0.2220375675826312, 0.19293113164997755], [0.2220375675826312, 0.19293113164997755]]
Recall for iterations:  [[0.4712086242659733, 0.43923926469519725], [0.4712086242659733, 0.43923926469519725], [0.4712086242659733, 0.43923926469519725]]


#### DATASET 3E

In [42]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3E-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3E-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3E-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_8972/3505167362.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3E-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8972/3505167362.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3E-Part3.csv')


In [43]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [44]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id604.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [45]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [46]:
global_model = build_model((24,1,1))

In [47]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [48]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [49]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id604.hdf5')

Epoch 1/5
51/51 [==============================] - 8s 154ms/step - loss: 0.2821 - accuracy: 0.9717 - val_loss: 1.9636 - val_accuracy: 0.6682
Epoch 2/5
51/51 [==============================] - 8s 157ms/step - loss: 0.0366 - accuracy: 0.9921 - val_loss: 0.7501 - val_accuracy: 0.6713
Epoch 3/5
51/51 [==============================] - 8s 162ms/step - loss: 0.0268 - accuracy: 0.9923 - val_loss: 0.6690 - val_accuracy: 0.6736
Epoch 4/5
51/51 [==============================] - 8s 159ms/step - loss: 0.0252 - accuracy: 0.9922 - val_loss: 0.5127 - val_accuracy: 0.6952
Epoch 5/5
51/51 [==============================] - 8s 157ms/step - loss: 0.0235 - accuracy: 0.9926 - val_loss: 0.4690 - val_accuracy: 0.7125
Epoch 1/5
51/51 [==============================] - 8s 164ms/step - loss: 0.0215 - accuracy: 0.9928 - val_loss: 0.3507 - val_accuracy: 0.7816
Epoch 2/5
51/51 [==============================] - 8s 157ms/step - loss: 0.0186 - accuracy: 0.9940 - val_loss: 0.6406 - val_accuracy: 0.6583
Epoch 3/5
51/

In [50]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.10486697405576706, 0.257686972618103], [0.11309099942445755, 0.36915287375450134], [0.1571233868598938, 0.5682361721992493]]
Accuracy for iterations:  [[0.9804259538650513, 0.9184407591819763], [0.9868605732917786, 0.8812577128410339], [0.9714715480804443, 0.8096890449523926]]
F1 for iterations:  [[0.9804387972989596, 0.9178054025404584], [0.9868637923234811, 0.8786637507403421], [0.9714436948455951, 0.7995370116182215]]
Precision for iterations:  [[0.9809291872122027, 0.9211076046577489], [0.9869095786733787, 0.8937045435044499], [0.9717799989701116, 0.8443418509629739]]
Recall for iterations:  [[0.9804259545999007, 0.9184407469301751], [0.9868605568377692, 0.881257719973843], [0.9714715592041836, 0.8096890212889631]]


#### DATASET 3F

In [51]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_8972/89903324.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8972/89903324.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8972/89903324.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part3.csv')


In [52]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [53]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id605.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [54]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [55]:
global_model = build_model((24,1,1))

In [56]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [57]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [58]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])



global_model.save('C:/Users/UX430/Documents/thesis/code/models/id605.hdf5')

Epoch 1/5
31/31 [==============================] - 5s 163ms/step - loss: 0.4862 - accuracy: 0.7824 - val_loss: 0.8133 - val_accuracy: 0.3810
Epoch 2/5
31/31 [==============================] - 5s 165ms/step - loss: 0.1971 - accuracy: 0.9281 - val_loss: 0.1752 - val_accuracy: 0.9654
Epoch 3/5
31/31 [==============================] - 5s 161ms/step - loss: 0.0932 - accuracy: 0.9769 - val_loss: 0.1095 - val_accuracy: 0.9902
Epoch 4/5
31/31 [==============================] - 5s 162ms/step - loss: 0.0620 - accuracy: 0.9845 - val_loss: 0.0872 - val_accuracy: 0.9960
Epoch 5/5
31/31 [==============================] - 5s 161ms/step - loss: 0.0482 - accuracy: 0.9877 - val_loss: 0.0772 - val_accuracy: 0.9981
Epoch 1/5
98/98 [==============================] - 16s 166ms/step - loss: 0.0217 - accuracy: 0.9929 - val_loss: 0.0198 - val_accuracy: 0.9898
Epoch 2/5
98/98 [==============================] - 17s 170ms/step - loss: 0.0162 - accuracy: 0.9945 - val_loss: 0.0143 - val_accuracy: 0.9938
Epoch 3/5
9

In [59]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.035965338349342346, 0.11802816390991211], [0.05962184816598892, 0.2757599651813507], [0.08898474276065826, 0.581328809261322]]
Accuracy for iterations:  [[0.9904686212539673, 0.9646152853965759], [0.9711356163024902, 0.8912482857704163], [0.9672937989234924, 0.7662573456764221]]
F1 for iterations:  [[0.9904723452407961, 0.9645994327783397], [0.9710852062318768, 0.8891216495116585], [0.9672131450905261, 0.7461594868219816]]
Precision for iterations:  [[0.9905988643460443, 0.9646079241040298], [0.9719703032510495, 0.902236437988811], [0.9687188040919812, 0.8243363146066571]]
Recall for iterations:  [[0.9904686084898771, 0.9646152728329579], [0.9711355867831372, 0.8912482743587881], [0.9672938151859535, 0.7662573566809562]]


#### DATASET 3G

In [60]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3G-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3G-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3G-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_8972/1702362103.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3G-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8972/1702362103.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3G-Part3.csv')


In [61]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [62]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id606.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [63]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [64]:
global_model = build_model((24,1,1))

In [65]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [66]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [67]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id606.hdf5')

Epoch 1/5
20/20 [==============================] - 4s 225ms/step - loss: 0.5101 - accuracy: 0.9398 - val_loss: 1.7044 - val_accuracy: 0.1459
Epoch 2/5
20/20 [==============================] - 3s 150ms/step - loss: 0.1858 - accuracy: 0.9526 - val_loss: 1.1748 - val_accuracy: 0.2292
Epoch 3/5
20/20 [==============================] - 3s 155ms/step - loss: 0.0778 - accuracy: 0.9779 - val_loss: 0.5421 - val_accuracy: 0.7749
Epoch 4/5
20/20 [==============================] - 3s 155ms/step - loss: 0.0545 - accuracy: 0.9818 - val_loss: 0.5352 - val_accuracy: 0.7419
Epoch 5/5
20/20 [==============================] - 3s 160ms/step - loss: 0.0458 - accuracy: 0.9844 - val_loss: 0.3217 - val_accuracy: 0.9157
Epoch 1/5
54/54 [==============================] - 9s 160ms/step - loss: 0.0236 - accuracy: 0.9918 - val_loss: 0.3693 - val_accuracy: 0.7411
Epoch 2/5
54/54 [==============================] - 9s 166ms/step - loss: 0.0201 - accuracy: 0.9931 - val_loss: 0.2382 - val_accuracy: 0.8541
Epoch 3/5
54/

In [68]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.09410139918327332, 0.3991059958934784], [0.13437721133232117, 0.9722410440444946], [0.11784116178750992, 0.9957458972930908]]
Accuracy for iterations:  [[0.9881168007850647, 0.8596781492233276], [0.9765549898147583, 0.7048971652984619], [0.9817113876342773, 0.6953788995742798]]
F1 for iterations:  [[0.9881193443176182, 0.8551147073415144], [0.9765309494289268, 0.6639681514001282], [0.9817037076793979, 0.6504287255082116]]
Precision for iterations:  [[0.9881532751605089, 0.8805236376187426], [0.9769034960094853, 0.7931737395189035], [0.9817804950266569, 0.7874536043719509]]
Recall for iterations:  [[0.9881168015425517, 0.8596781225023614], [0.9765549680095825, 0.7048971881130568], [0.9817114142978176, 0.6953789144808544]]


### Experimenting with epochs and iterations 

Use of dataset 3C, considering it contains an equal partition of the data between the 3 nodes. 

#### #Epochs = 10, #Iterations = 10

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id610.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])



global_model.save('C:/Users/UX430/Documents/thesis/code/models/id610.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### #Epochs = 5, #Iterations = 5

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id611.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id611.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)